In [41]:
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2.12.0
1.4.0


In [42]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

In [43]:
option=2
#_model='binary_xgb_tf_model'   # the is resonable because unsseen and tune 70%
_model="tuned_binary_xgb_tf_model"  # best 75% on unseen test
model_version=f'{_model}_v1_t150723'

model_gs_path=f"gs://demo-tuned-tf-incident-pongthorn/{_model}"
print(model_gs_path)

projectId="pongthorn"
dataset_id="SMartML"


prediction_table="new2_result_binary_prediction_incident"
if option==1:
    unusedCols_unseen=['id','severity_name','imported_at','open_to_close_hour']
else:
    unusedCols_unseen=['id','severity_name','imported_at','range_open_to_close_hour']
    

gs://demo-tuned-tf-incident-pongthorn/tuned_binary_xgb_tf_model


# Load Model

In [44]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-07-20 16:34:42.1853 UTC kernel.cc:1243] Loading model from path gs://demo-tuned-tf-incident-pongthorn/tuned_binary_xgb_tf_model/assets/ with prefix fe39250a4ca44dae


Model: "gradient_boosted_trees_model_1"


[INFO 23-07-20 16:34:42.6030 UTC decision_forest.cc:660] Model loaded with 60 root(s), 4704 node(s), and 6 input feature(s).
[INFO 23-07-20 16:34:42.6039 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-07-20 16:34:42.6044 UTC kernel.cc:1075] Use fast generic engine


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# Make Prediction

In [45]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
df=query_result.to_dataframe()
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        40 non-null     Int64         
 1   severity_id               40 non-null     Int64         
 2   severity_name             40 non-null     object        
 3   sla                       40 non-null     object        
 4   product_type              40 non-null     object        
 5   brand                     40 non-null     object        
 6   service_type              40 non-null     object        
 7   incident_type             40 non-null     object        
 8   open_to_close_hour        40 non-null     float64       
 9   range_open_to_close_hour  40 non-null     object        
 10  imported_at               40 non-null     datetime64[ns]
dtypes: Int64(2), datetime64[ns](1), float64(1), object(7)
memory usage: 3.6+ KB
None


In [46]:
unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   severity_id         40 non-null     Int64  
 1   sla                 40 non-null     object 
 2   product_type        40 non-null     object 
 3   brand               40 non-null     object 
 4   service_type        40 non-null     object 
 5   incident_type       40 non-null     object 
 6   open_to_close_hour  40 non-null     float64
dtypes: Int64(1), float64(1), object(5)
memory usage: 2.4+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,open_to_close_hour
30,3,24x7 4Hrs Response Time,Firewall,CheckPoint,Incident,Network Adapter Failure,284.183333
31,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2.200000
32,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,4.500000
33,4,24x7 4Hrs Resolution Time,Software,Veeam,Request,Software,4.000000
34,4,24x7 4Hrs Resolution Time,Software,Veeam,Request,Software,2.000000
35,4,24x7 4Hrs Response Time,Storage,NetApp,Request,OS / Firmware,3060.216667
36,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,14.500000
37,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.000000
38,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.500000
39,4,24x7 4Hrs Response Time,Firewall,Palo Alto,Request,Software,622.283333


In [47]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>


In [48]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    # prob = tf.nn.sigmoid(predResult) # no need to convert to Signmoid
    # print(prob)
    _class= 1 if predResult[0]>=0.5 else 0  
    # pred_seveirty_id= "critical" if _class==1 else "normal"
    predServerityIDList.append(_class) #0=normal , 1=critical
    print(f"{predResult} : {_class}")

dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_binary_severity"])   


1/1 [==============================] - 0s 64ms/step
[0.25636742] : 0
[0.17903766] : 0
[0.9165269] : 1
[0.9285503] : 1
[0.87518704] : 1
[0.69971377] : 1
[0.9147515] : 1
[0.92415375] : 1
[0.84933454] : 1
[0.44458163] : 0
[0.9435532] : 1
[0.8062814] : 1
[0.7003334] : 1
[0.9286715] : 1
[0.5661504] : 1
[0.66304684] : 1
[0.9765022] : 1
[0.951737] : 1
[0.44791013] : 0
[0.9255828] : 1
[0.42805284] : 0
[0.45431378] : 0
[0.20582551] : 0
[0.76085263] : 1
[0.5264448] : 1
[0.53275955] : 1
[0.67538774] : 1
[0.19062455] : 0
[0.1809187] : 0
[0.7941052] : 1
[0.9313895] : 1
[0.2712011] : 0
[0.05013854] : 0
[0.03604104] : 0
[0.08623972] : 0
[0.01716065] : 0
[0.01691318] : 0
[0.01121346] : 0
[0.00765095] : 0
[0.02547074] : 0


2023-07-20 16:34:46.188921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [40]
	 [[{{node Placeholder/_0}}]]


In [49]:
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df['label_binary_severity'] =df['severity_id'].apply(map_4to2_serverity)

dfPred
df=pd.concat([df,dfPred],axis=1)
df

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,range_open_to_close_hour,imported_at,label_binary_severity,pred_binary_severity
0,3198,2,Major,24x7 6Hrs Resolution Time,Software,YIP,Incident,General Incident,1.600000,soonest,2023-07-20 02:50:49.351582,1,0
1,3195,2,Major,24x7 6Hrs Resolution Time,Software,YIP,Incident,General Incident,230.000000,fair,2023-07-20 02:50:49.351582,1,0
2,3125,2,Major,9x5 NBD 4Hrs Response Time,Server,HPE,Incident,OS / Firmware,161.500000,soon,2023-07-20 02:50:49.351582,1,1
3,3171,2,Major,24x7 4Hrs Response Time,Server,HPE,Incident,System Board Failure,59.266667,soon,2023-07-20 02:50:49.351582,1,1
4,3131,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Battery Failure,85.283333,soon,2023-07-20 02:50:49.351582,1,1
5,3182,2,Major,24x7 4Hrs Resolution Time,Storage,HPE,Incident,Hard Disk Drive Failure,6.233333,soonest,2023-07-20 02:50:49.351582,1,1
6,3139,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,6.333333,soonest,2023-07-20 02:50:49.351582,1,1
7,3161,2,Major,24x7 4Hrs Response Time,Server,DELL,Incident,Network Card Failure,17.400000,soonest,2023-07-20 02:50:49.351582,1,1
8,3176,2,Major,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,7.833333,soonest,2023-07-20 02:50:49.351582,1,1
9,2856,2,Major,8x5 4Hrs Response Time,Storage,NetApp,Incident,General Incident,1177.666667,latest,2023-07-20 02:50:49.351582,1,0


# Evaluate model

In [50]:
from sklearn.metrics import confusion_matrix,classification_report
className=list(set().union(list(df['pred_binary_severity'].unique()),list(df['label_binary_severity'].unique())))
print(className)
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'pred-{x}' for x in className]
y_true=list(df['label_binary_severity'])
y_pred=list(df['pred_binary_severity'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
print(cm_df)
print(classification_report(y_true, y_pred, labels=className))

[0, 1]
          pred-0  pred-1
actual-0      15       7
actual-1       3      15
              precision    recall  f1-score   support

           0       0.83      0.68      0.75        22
           1       0.68      0.83      0.75        18

    accuracy                           0.75        40
   macro avg       0.76      0.76      0.75        40
weighted avg       0.77      0.75      0.75        40



# Write Prediction Result to BQ

In [51]:
df=df[['id','label_binary_severity','pred_binary_severity']]
df['prediction_datetime']=datetime.now()
df['model_version']=model_version
print(df)

      id  label_binary_severity  pred_binary_severity  \
0   3198                      1                     0   
1   3195                      1                     0   
2   3125                      1                     1   
3   3171                      1                     1   
4   3131                      1                     1   
5   3182                      1                     1   
6   3139                      1                     1   
7   3161                      1                     1   
8   3176                      1                     1   
9   2856                      1                     0   
10  3048                      1                     1   
11  3063                      1                     1   
12  3193                      1                     1   
13  3148                      1                     1   
14  3194                      1                     1   
15  3178                      1                     1   
16  3186                      1

/var/tmp/ipykernel_5945/3090584964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction_datetime']=datetime.now()
/var/tmp/ipykernel_5945/3090584964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['model_version']=model_version


In [53]:
predictResult_table_id=f"{projectId}.{dataset_id}.{prediction_table}"

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("label_binary_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_binary_severity", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") ,
    bigquery.SchemaField("model_versoin", "STRING", mode="REQUIRED")     
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    # table.time_partitioning = bigquery.TimePartitioning(
    # type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Created table pongthorn.SMartML.new2_result_binary_prediction_incident


In [54]:
def loadDataFrameToBQ():
    # WRITE_TRUNCATE , WRITE_APPEND
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  40 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'